In [ ]:
#Imports
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from math import pi
from time import time
import minterpy_levelsets as ls

# Local imports
import surfgeopy as sp

# Gauss Bonnet theorem on torus
This is a benchmark of computing surface integrals using high-order surface quadrature (HOSQ) method and GPLS method for algebraic varieties.



# Step 1: Load and extract the vertices of a torus mesh composed of          triangles.

In [ ]:
mesh_path="../meshes/torus_N=2512.mat"
mesh_mat = scipy.io.loadmat(mesh_path)
pointcloud= mesh_mat["xs"]

In [ ]:
R=0.5
r=0.3
def phi(x: np.ndarray):
    ph = np.sqrt(x[0]*x[0] + x[1]*x[1])
    return (ph - R)*(ph - R) + x[2]*x[2] - r*r
def dphi(x: np.ndarray):
    ph = np.sqrt(x[0]*x[0] + x[1]*x[1])
    return np.array([-2*R*x[0]/ph + 2*x[0],-2*R*x[1]/ph + 2*x[1],2*x[2]])

# Step 2: Perform surface fitting

In [ ]:
result = np.zeros(pointcloud.shape[0])  # initialize an empty array to store the results
for i in range(pointcloud.shape[0]):
    result[i] = phi(pointcloud[i])
print(f"The accuracy of the given mesh is:{(result).max()}")
if (result).max()>1e-8:
    for i in range(pointcloud.shape[0]):
        pointcloud[i,:]=sp.SimpleImplicitSurfaceProjection(phi, dphi,pointcloud[i,:])
newt_poly = ls.LevelsetPoly(pointcloud, method='BK',verbose=True)

# Step 3: We execute  HOSQ for the torus $\mathbb{T}^2_{r,R}$.


This step involves utilizing the zero level set provided by GPLS and computing the curvature with the help of GPLS:

If $M$ is orienatble, then  the gradient $\nabla Q_M = (\partial_x Q_M,\, \partial_y Q_M,\, \partial_z Q_M) \in \mathbb{R}^3$ never vanishes on $M$ and provides, together with the Hessian $H_M =\nabla(\nabla Q_M) \in \mathbb{R}^{3 \times 3}$ of $Q_M$
\begin{equation*}
 H_M  = \left(\begin{array}{ccc}
 \frac{\partial^2 Q_M}{\partial_{x}^2} & \frac{\partial^2 Q_M}{\partial_{x}\partial_y} & \frac{\partial^2 Q_M}{\partial_{x}\partial_z} \\
 \frac{\partial^2 Q_M}{\partial_{y}\partial_x} & \frac{\partial^2 Q_M}{\partial_y^2} & \frac{\partial^2 Q_M}{\partial_{y}\partial_z} \\
 \frac{\partial^2 Q_M}{\partial_z\partial_{x}} & \frac{\partial^2 Q_M}{\partial_{z}\partial_y} & \frac{\partial^2 Q_M}{\partial_z^2} \\
 \end{array}\right) ,
\end{equation*}
the main ingredients for the following computations. Both Gauss and mean curvature can be computed from these quantities \cite{goldman2005} as:
\begin{align}
  K_{\mathrm{Gauss}} &= \frac{\det \left(\begin{array}{cc}H_M  & \nabla Q_M^T \\  \nabla Q_M & 0 \end{array}\right)}{\|\nabla Q_M\|^4} \label{eq:GC}\\
  K_{\mathrm{mean}} &= \frac{\nabla Q_M H_M \nabla Q_M^T - \|\nabla Q_M\|^2\mathrm{trace}(H_M)}{2\|\nabla Q_M\|^3}\,. \label{eq:MC}
\end{align}

In [ ]:
def err_t(intp_degree,lp_degree,func,grad_fun,mesh_path,Refinement):
    mesh_mat = scipy.io.loadmat(mesh_path)
    xs =mesh_mat["xs"]
    elems = mesh_mat["surfs"] - 1
    t0 = time()
    pnts, ws, offset = sp.compute_surf_quadrature(func,grad_fun, xs, elems,intp_degree,lp_degree,Refinement)
    # number of quadrature points
    nqp = pnts.shape[0]
    #compute the mean and Gauss curvature using GPLS
    mean_curvature, gauss_curvature = newt_poly.compute_curvatures_at(pnts)
    # function value at each quadrature points
    fs_qp = np.array(
            [
                    gauss_curvature[qpid]

                for qpid in range(nqp)
            ]
        )
    # numerical integrations
    nints = np.matmul(ws, fs_qp)
    exact_area =0
    t1 = time()
    print("Relative error: ", abs( nints - exact_area))
    print ("The main function takes:",{(t1-t0)})
    error=abs( nints - exact_area)
    return error

# here is the error obtained using Dune Curved Grid
error_dune2_14=np.array([9.262908e-05, 4.163018e-05, 6.594602e-08, 1.587536e-07,
                         5.133794e-10, 9.538793e-10, 1.248652e-11, 9.704872e-11, 
                         1.370585e-09, 1.559943e-08, 1.181970e-07,3.655481e-07, 4.165369e-06])

eror_HOSQ_GPLS=np.array([6.0068600790345785e-05,1.4799681049813568e-05,4.8092809326518576e-08,
                        8.489890112109433e-09,1.9726209229443958e-10,7.845483377538764e-12,
                        2.545225315231381e-13,1.8339431535896455e-13,6.941821249772939e-14,
                       7.100700236128965e-14,7.192987525050931e-14,5.520822514426538e-14,5.989306273157524e-14])
# Degree of Polynomial
Nrange = list(range(2,15))
lp_degree=float("inf")
refinement=0
error1=[] 
for n in Nrange:
    if n%1==0:print(n)
    erro1 = err_t(int(n),lp_degree,newt_poly,newt_poly.compute_gradients_at,mesh_path,refinement)
    error1.append(erro1)

plt.semilogy(Nrange, eror_HOSQ_GPLS, '-or')
plt.semilogy(Nrange, error_dune2_14, '-ob')
plt.xlabel("Degree of Polynomial",fontsize=13)
plt.ylabel("Relative Error",fontsize=13)
plt.legend(['HOSQ+GPLS','DCG'],prop={'size': 13},loc='upper center')
plt.xticks(np.arange(min(Nrange), max(Nrange)+1, 1.0))
plt.ylim([2.758195177427762e-16,3.9514540203871754e-04])
plt.grid()
plt.savefig("../images/dune_vs_HOSQ+GPLS_Torus.pdf")